In [1]:


import dsx_core_utils, requests, jaydebeapi, os, io, sys
from pyspark.sql import SparkSession
import pandas as pd
df1 = None
dataSet = dsx_core_utils.get_remote_data_set_info('customers')
dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
if (sys.version_info >= (3, 0)):
  conn = jaydebeapi.connect(dataSource['driver_class'], dataSource['URL'], [dataSource['user'], dataSource['password']])
else:
  conn = jaydebeapi.connect(dataSource['driver_class'], [dataSource['URL'], dataSource['user'], dataSource['password']])
query = 'select * from "' + (dataSet['schema'] + '"."' if (len(dataSet['schema'].strip()) != 0) else '') +  dataSet['table'] + '"'

if (dataSet['query']):
    query = dataSet['query']
df1 = pd.read_sql(query, con=conn)
df1.head()




CustomerID  Retire Invest Educ Mortgage LOC  GENDER    MARITAL CHILDREN  \
0         602     0.0     No   No       No  No    Male  Separated       No   
1         603     0.0     No   No      Yes  No    Male    Married       No   
2         604     0.0     No   No      Yes  No  Female  Separated       No   
3         605     0.0     No   No      Yes  No  Female    Married       No   
4         606     0.0     No   No       No  No    Male    Married       No   

  WORKING HighMonVal  AgeRange  
0     Yes        Yes  31 to 40  
1     Yes        Yes  31 to 40  
2     Yes        Yes  23 to 30  
3     Yes        Yes  17 to 22  
4     Yes         No  23 to 30

In [2]:


import dsx_core_utils, requests, os, io
from pyspark.sql import SparkSession
# Add asset from remote connection
df2 = None
dataSet = dsx_core_utils.get_remote_data_set_info('customers')
dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
sparkSession = SparkSession(sc).builder.getOrCreate()
# Load JDBC data to Spark dataframe
dbTableOrQuery = '"' + (dataSet['schema'] + '"."' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table'] + '"'
if (dataSet['query']):
    dbTableOrQuery = "(" + dataSet['query'] + ") TBL"
df2 = sparkSession.read.format("jdbc").option("url", dataSource['URL']).option("dbtable", dbTableOrQuery).option("user",dataSource['user']).option("password",dataSource['password']).load()
df2.show(5)




+----------+------+------+----+--------+---+------+---------+--------+-------+----------+--------+
|CustomerID|Retire|Invest|Educ|Mortgage|LOC|GENDER|  MARITAL|CHILDREN|WORKING|HighMonVal|AgeRange|
+----------+------+------+----+--------+---+------+---------+--------+-------+----------+--------+
|       602|     0|    No|  No|      No| No|  Male|Separated|      No|    Yes|       Yes|31 to 40|
|       603|     0|    No|  No|     Yes| No|  Male|  Married|      No|    Yes|       Yes|31 to 40|
|       604|     0|    No|  No|     Yes| No|Female|Separated|      No|    Yes|       Yes|23 to 30|
|       605|     0|    No|  No|     Yes| No|Female|  Married|      No|    Yes|       Yes|17 to 22|
|       606|     0|    No|  No|      No| No|  Male|  Married|      No|    Yes|        No|23 to 30|
+----------+------+------+----+--------+---+------+---------+--------+-------+----------+--------+
only showing top 5 rows

